## Pradedam nuo nurodymo į duomenis (photo). Vienas aplanakas jame 6 nuotraukos tyrimo objekto is 6 nuotraukos kalibracinės panelės.

In [ ]:
## auto perkraunam variables pradedam iš naujo
%load_ext autoreload
%autoreload 2

Užkraunam visiškai visas reikalingas bibliotekas

In [ ]:
#Įtraukiam visas reikalingas bibliotekas
import micasense.image as image
import micasense.capture as capture
import micasense.panel as panel
import matplotlib.pyplot as plt
import numpy as np
import os, glob
from pathlib import Path

Duomenų masyvai 1D su 6 pavadinimais

In [ ]:
#folderis kur visos foto ir tyrimo ir kalibracijos, nuotraukų setų pavadinimai
imagePath = Path("../data/tabakas_hand")

#tyriamo objekto nuotraukų setas IMG_****_*.tif micasense pavadinimo formatas
imageNames = list(imagePath.glob('IMG_0015_*.tif'))
imageNames = [x.as_posix() for x in imageNames]

#kalibracinės paneles objekto nuotraukų setas IMG_****_*.tif micasense pavadinimo formatas
panelNames = list(imagePath.glob('IMG_0002_*.tif'))
panelNames = [x.as_posix() for x in panelNames]

#patikrinimas
print(f"Tyriamo objekto nuotraukų setas ({len(imageNames)}):")
for name in imageNames: print(f" - {os.path.basename(name)}")
#patikrinimas
print(f"\nKalibracinės panelės nuotraukų setas ({len(panelNames)}):")
for name in panelNames: print(f" - {os.path.basename(name)}")

sukuriam "capture" objektus, vienas tyrymo obejekto fotografijom antras kalibracinės panelės fotografijom

In [ ]:
#du capture objektai, tai ir yra visi duomenys sugrupuoti/nurodyti "capture" obejktuose

#prisilaikom thecapture pavadinimo tokį naudoja micasence bibliotekas kūrėjai
thecapture = capture.Capture.from_filelist(imageNames)
#prisilaikom panelCap pavadinimo tokį naudoja micasence bibliotekas kūrėjai
panelCap = capture.Capture.from_filelist(panelNames)

#kameros duomenys, kamera turim tik viena bet d4l pasitikrinimo paliekam
cam_model = thecapture.camera_model
cam_serial = thecapture.camera_serial

#pasitikrinam ką realiai nuskaitėm
print("Camera model:",cam_model)
print("Bit depth:", thecapture.bits_per_pixel)
#print("Camera serial number:", cam_serial)
print("Capture 'image' ID:",thecapture.uuid)
print("Capture 'panel' ID:",panelCap.uuid)



Irradiance. atspindžiai

In [ ]:
#paneles foto pavertimas atspindžių skaičių masyvu(atspindžių nuotrauka, pikselių reikšmės perskaičiuojam į atspindžių vertes 0.0 - 1.0 kiekvieno kiekvieno pixelio atskirai)
irradiance_list = panelCap.panel_irradiance(panelCap.panel_albedo())

thecapture.compute_reflectance(irradiance_list)
print("Reflectance computed for all survey bands.")

print("--- Current Irradiance Values ---")
for name, value in zip(thecapture.eo_band_names(), irradiance_list):
    print(f"{name:20}: {value:.2f} W/m²/nm")

thecapture.plot_undistorted_reflectance(irradiance_list)


In [ ]:
# 1. Select the band (0 = Blue usually, check your metadata)

img = thecapture.images[0]



# 3. Plot it
#plt.figure(figsize=(10, 8))
#plt.imshow(reflectance_data, cmap='viridis', vmin=0, vmax=1.0)
#plt.imshow(img.undistorted_reflectance())
#plt.colorbar(label='Reflectance')
#plt.title(f"Band {band_idx}: {img.band_name} - Reflectance Map")
#plt.axis('off')
#plt.show()


#fig, axes = plt.subplots(1, 5, figsize=(20, 5))


for ax, cmap in zip(axes, cmaps):
    ax.imshow(data, cmap=cmap, vmin=0, vmax=0.6)
    ax.set_title(cmap)
    ax.axis('off')


for i, img in enumerate(thecapture.images):
    img = thecapture.images[i]
    plt.imshow(img.undistorted_reflectance())
    plt.show()


fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel() # Flatten to 1D for the loop

for i, img in enumerate(thecapture.images):
    axes[i].imshow(img.undistorted_reflectance(), cmap='viridis', vmin=0, vmax=0.8)
    axes[i].set_title(f"Band {i}: {img.band_name}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
#atvaizduosim "lentele" bet tik viena eilutė paimam tą gabaliuką nuotraukos, kurį kameros softas nustatė: kalibracinės panelės paviršius
fig, axes = plt.subplots(1, 6, figsize=(12, 12))
axes = axes.ravel() # Flatten to a 1D list for easy looping

for i, img in enumerate(panelCap.images):
     
    p = panel.Panel(img)

    # 1. Calculate Raw Mean
    pts = np.array(p.panel_corners())
    x1, y1 = pts.min(axis=0).astype(int)
    x2, y2 = pts.max(axis=0).astype(int)
    crop = img.raw()[y1:y2, x1:x2]
    actual_mean = np.mean(crop)
   
    # 2. Get corresponding Irradiance from your list
    # We use the index 'i' to match the current band
    irr_val = irradiance_list[i] if i < len(irradiance_list) else 0.0

    ax = axes[i]
#    print(f"--- {img.band_name} ---")
#    print(f"Calculated Mean Radiance: {actual_mean:.2f} (Raw DN)")
#    print(f"Irradiance              : {irr_val:.2f} W/m²/nm")
    ax.imshow(crop, cmap='gray')
    ax.set_title(f"Band No.: {i} {img.band_name}\nMean Radiance: {actual_mean:.0f}\nIrradiance: {irr_val:.2f}", fontsize=10, ha='left', loc='left')
    ax.axis('off')



plt.tight_layout()
plt.show()